In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics.pairwise import cosine_similarity

import keras
from tensorflow import keras
from keras import Sequential
from keras import layers
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout,SeparableConv2D, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

import librosa
import librosa.display
import IPython.display as ipd

import eli5
from eli5.sklearn import PermutationImportance

import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
dir_ = './data'
print(list(os.listdir(f'{dir_}/genres_original/')))


['pop', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']


In [4]:
y, s = librosa.load(f'{dir_}/genres_original/blues/blues.00023.wav')

print('y:', y, '\n')
print('y shape:', np.shape(y), '\n')
print('Sample Rate (KHz):', s, '\n')

print('Check Len of Audio:', 661794/22050)

y: [ 0.17184448  0.20730591  0.08227539 ...  0.00271606 -0.02062988
 -0.01370239] 

y shape: (661794,) 

Sample Rate (KHz): 22050 

Check Len of Audio: 30.013333333333332


In [5]:
audio, _ = librosa.effects.trim(y)

print('Audio File:', audio, '\n')
print('Audio File shape:', np.shape(audio))

Audio File: [ 0.17184448  0.20730591  0.08227539 ...  0.00271606 -0.02062988
 -0.01370239] 

Audio File shape: (661794,)


# Modeling   

In [6]:
df = pd.read_csv(f'{dir_}/features_3_sec.csv')
df = df.iloc[0:, 1:] 
df


,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,3015.559458,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,3746.694524,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,2442.362154,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,4313.266226,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


In [7]:
y = df['label']
X = df.loc[:, df.columns != 'label']

cols = X.columns
scaler = preprocessing.MinMaxScaler()
np_scaled = scaler.fit_transform(X)

X = pd.DataFrame(np_scaled, columns = cols)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [9]:
def model_assess(model, title = "Default"):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(confusion_matrix(y_test, preds))
    print('Accuracy', title, ':', round(accuracy_score(y_test, preds), 5), '\n')

In [11]:
from joblib import dump, load

In [12]:
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, "KNN")

dump(knn, 'knn.joblib')

[[256   5  22   4   2   8   6   0  10   6]
 [  0 297   1   0   0  10   0   0   0   0]
 [  9   2 230  11   1  13   0   1  12   7]
 [  7   6   8 246   8   0   2   4   6  14]
 [  7   1  15  24 217   0   9  21  14   3]
 [  2  30  16   2   0 233   0   2   1   0]
 [  6   0   2  15   5   0 252   0   4  19]
 [  0   0   6  17   2   2   0 229  10   1]
 [  3   1  16  20  13   0   2  11 248   2]
 [ 11   1  20  35   0   7   7   2  10 207]]
Accuracy KNN : 0.80581 



['knn.joblib']

In [13]:
tree = DecisionTreeClassifier()
model_assess(tree, "Decission trees")

dump(tree, 'tree.joblib')

[[173   0  36  20   7  15  13   7  20  28]
 [  4 257  14   0   0  25   1   1   1   5]
 [ 29   3 142  12  11  28   9   6  18  28]
 [  9   3  11 176  25   6   7  18  16  30]
 [  6   1  15  28 204   3  15  18  20   1]
 [ 11  14  17   7   0 211   3   7   2  14]
 [  9   0   6  19   9   2 236   2   6  14]
 [  0   1  15  10  15   3   1 205   7  10]
 [ 12   3  23  20  29   5   5  12 186  21]
 [ 17   4  30  22  15  21  28   6  19 138]]
Accuracy Decission trees : 0.64331 



['tree.joblib']

In [14]:
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, "Random Forest")

dump(rforest, 'rforest.joblib')

[[246   1  31   8   4   8  15   0   6   0]
 [  0 294   1   0   0  12   0   0   0   1]
 [ 13   1 212  12   0  23   3   3   9  10]
 [  1   4  11 235  14   1   5   8  10  12]
 [  4   0   7   8 250   2  13  15  10   2]
 [  7  21   5   1   0 251   0   0   1   0]
 [  1   0   1   2   3   1 281   0   6   8]
 [  0   0  12   4   2   1   0 238   5   5]
 [  5   2   9  11   7   4   4  17 254   3]
 [  7   2  29  34   1  12  19   4  13 179]]
Accuracy Random Forest : 0.81415 



['rforest.joblib']

In [15]:
svm = SVC(decision_function_shape="ovo")
model_assess(svm, "Support Vector Machine")

dump(svm, 'svm.joblib')

[[241   5  10   9   2  12  24   0   7   9]
 [  0 294   2   0   0  10   0   0   0   2]
 [ 19   0 192  18   0  23   1   5   5  23]
 [  2   6   7 199  18   7  10  12  14  26]
 [  7   2   7  18 220   0   8  16  28   5]
 [ 12  25   9   2   0 234   0   1   2   1]
 [ 10   0   0   7   4   0 270   0   2  10]
 [  0   1  10   8   4   1   0 233   4   6]
 [ 13   2  16  18  20   3   5  10 220   9]
 [ 19   2  24  45   7  11  16   9  10 157]]
Accuracy Support Vector Machine : 0.75409 



['svm.joblib']

In [16]:
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
model_assess(lg, "Logistic Regression")

dump(lg, 'lg.joblib')

[[212   5  12   7   4  18  40   0  11  10]
 [  2 287   5   0   0  10   0   0   0   4]
 [ 23   1 166  18   2  21   1  19   9  26]
 [  3   5  15 182  23   3  16  21  15  18]
 [  8   2   8  17 201   0  15  24  31   5]
 [ 11  28  14   5   1 214   0   2   7   4]
 [ 10   0   0   7   4   0 266   0   3  13]
 [  0   0  14   8  11   0   0 226   3   5]
 [ 14   1  19  12  29   5   8  10 210   8]
 [ 35   3  25  32   5  14  22  18  19 127]]
Accuracy Logistic Regression : 0.6977 



['lg.joblib']

In [17]:
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1)
model_assess(nn, "Neural Nets")

dump(nn, 'nn.joblib')

[[185   1  50   3   4  19  24   0  12  21]
 [  2 281   3   1   0  20   0   0   1   0]
 [ 21   0 164  11   1  28   0   6  12  43]
 [  3   4  10 171  27   4  10  14  19  39]
 [  6   1   4  21 203   3  12  17  38   6]
 [  8  22  21   0   0 217   1   2   1  14]
 [ 12   0   1   8   6   0 261   0   0  15]
 [  0   0   4  10   5   0   0 234  12   2]
 [  4   1  32  17  29   1   2   9 216   5]
 [ 20   2  54  48   2  18  18   9  11 118]]
Accuracy Neural Nets : 0.68402 



In [19]:
ada = AdaBoostClassifier(n_estimators=1000)
model_assess(ada, "AdaBoost")

dump(ada, 'ada.joblib')

[[ 81   3  28  14  19  14 120   0  13  27]
 [  1 285   5   1   1   7   0   0   1   7]
 [ 45   7  93  15  38  11  15  31  18  13]
 [  7   6  16  56  41   3  65  75  12  20]
 [  3   3   7  20  88   2  97  75  11   5]
 [ 14  61  23  11  13  97  12  22  15  18]
 [  5   0   6   6   2   1 274   0   0   9]
 [  1   1   5   5   6   0   1 240   4   4]
 [  9   4  29  30  62   2  21  66  70  23]
 [ 14   5  32  45  22   3  69  46   3  61]]
Accuracy AdaBoost : 0.44878 



['ada.joblib']

In [4]:
def get_features(y, sr):
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    harmony, perceptr = librosa.effects.hpss(y)
    tempo = librosa.beat.tempo(y=y, sr=sr)
    mfcc1 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=1)
    mfcc2 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=2)
    mfcc3 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=3)
    mfcc4 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=4)
    mfcc5 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=5)
    mfcc6 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=6)
    mfcc7 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=7)
    mfcc8 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=8)
    mfcc9 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=9)
    mfcc10 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=10)
    mfcc11 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=11)
    mfcc12 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)
    mfcc13 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc14 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=14)
    mfcc15 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=15)
    mfcc16 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=16)
    mfcc17 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=17)
    mfcc18 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=18)
    mfcc19 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=19)
    mfcc20 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)


    return [np.mean(chroma_stft), np.var(chroma_stft), np.mean(rmse), np.var(rmse), np.mean(spec_cent), np.var(spec_cent), np.mean(spec_bw), np.var(spec_bw), np.mean(rolloff), np.var(rolloff), np.mean(zcr), np.var(zcr), np.mean(harmony), np.var(harmony), np.mean(perceptr), np.var(perceptr), tempo[0],
     np.mean(mfcc1), np.var(mfcc1),
        np.mean(mfcc2), np.var(mfcc2),
        np.mean(mfcc3), np.var(mfcc3),
        np.mean(mfcc4), np.var(mfcc4),
        np.mean(mfcc5), np.var(mfcc5),
        np.mean(mfcc6), np.var(mfcc6),
        np.mean(mfcc7), np.var(mfcc7),
        np.mean(mfcc8), np.var(mfcc8),
        np.mean(mfcc9), np.var(mfcc9),
        np.mean(mfcc10), np.var(mfcc10),
        np.mean(mfcc11), np.var(mfcc11),
        np.mean(mfcc12), np.var(mfcc12),
        np.mean(mfcc13), np.var(mfcc13),
        np.mean(mfcc14), np.var(mfcc14),
        np.mean(mfcc15), np.var(mfcc15),
        np.mean(mfcc16), np.var(mfcc16),
        np.mean(mfcc17), np.var(mfcc17),
        np.mean(mfcc18), np.var(mfcc18),
        np.mean(mfcc19), np.var(mfcc19),
        np.mean(mfcc20), np.var(mfcc20)]

In [5]:
y, sr = librosa.load(f'{dir_}/genres_original/blues/blues.00000.wav')
y, _ = librosa.effects.trim(y)


f = get_features(y, sr)

test = [0.35008811950683594, 0.08875656872987747 , 0.1302279233932495   , 0.0028266964945942163 , 1784.165849538755     , 129774.06452515082   , 2002.4490601176963     , 85882.76131549841     , 3805.8396058403423, 901505.4255328419 , 0.08304482066898686    , 0.0007669456545940504 , -4.5297241740627214e-05, 0.00817228201776743   , 7.783231922076084e-06  , 0.00569818215444684   , 123.046875        , -113.57064819335938, 2564.20751953125  , 121.57179260253906, 295.913818359375  , -19.168142318725586 , 235.57443237304688, 42.36642074584961   , 151.10687255859375, -6.364664077758789   , 167.93479919433594, 18.623498916625977  , 89.18083953857422 , -13.704891204833984  , 67.66049194335938 , 15.34315013885498    , 68.93257904052734 , -12.274109840393066  , 82.2042007446289  , 10.976572036743164   , 63.38631057739258 , -8.326573371887207    , 61.773094177246094, 8.803791999816895    , 51.24412536621094 , -3.672300100326538   , 41.21741485595703 , 5.747994899749756    , 40.55447769165039 , -5.162881851196289   , 49.775421142578125, 0.752740204334259    , 52.4209098815918  , -1.6902146339416504  , 36.524070739746094, -0.4089791774749756  , 41.597103118896484, -2.3035225868225098  , 55.062923431396484, 1.2212907075881958   , 46.93603515625]







ValueError: Expected 2D array, got 1D array instead:
array=[ 3.50128502e-01  8.87716636e-02  1.30184323e-01  2.82838009e-03
  1.78412264e+03  1.29745485e+05  2.00241241e+03  8.58344104e+04
  3.80572303e+03  9.01252917e+05  8.30448207e-02  7.66945655e-04
 -4.90663006e-05  8.17246269e-03 -1.06184834e-05  5.70443785e-03
  1.23046875e+02 -1.13598824e+02  2.56936938e+03  3.98592186e+00
  1.52587793e+04 -3.73013449e+00  1.03701230e+04  7.79337788e+00
  8.21372656e+03  4.96225214e+00  6.63664111e+03  7.23886633e+00
  5.57131104e+03  4.24763584e+00  4.83875195e+03  5.63415670e+00
  4.25599023e+03  3.64432740e+00  3.82391309e+03  4.37698936e+00
  3.45268774e+03  3.22216654e+00  3.15775830e+03  3.68716002e+00
  2.90126343e+03  3.12122917e+00  2.68510327e+03  3.30861807e+00
  2.49666064e+03  2.74385810e+00  2.33800073e+03  2.61934876e+00
  2.19538501e+03  2.36598372e+00  2.06942114e+03  2.21183300e+00
  1.95716833e+03  1.97422719e+00  1.85807300e+03  1.93663919e+00
  1.76754333e+03].
Reshape your